# Final Project - DFT calculations

## Part 2: NEB calculations of oxygen vacancy mobility

In this part, you will be calculation the oxygen vacancy mobility in your perovskite as a fuel cell cathode. We will consider two directions of diffusion in the perovskite, denoted *y* and _diag_, see the figure below.

<img src="diffusion_directions.png" alt="Drawing" style="width: 200px;"/>


The work flow is as follows:
- The NEB calculations will be done in a 2x2x2 repeated unit cell.
- Create the initial image by removing an oxygen
    - Rattle atoms randomly to break symmetries (A higher value in the .rattle() moves the atoms more)
    - Relax. Due to the rattle, we have to relax it quite tightly.
- Construct the _y_-direction final image
    - Repeat same steps as for the initial image
    - Your initial and final state images are (probably) not going to have identical energies, but they should be close. Remember to check! The difference should be no larger than 0.1 eV.
        - If the differences for some reason become too large, try relaxing the structure with a lower fmax, or rattle the atoms less.
- Calculate the MEP using NEB in the _y_-direction
- Do the same for the diagonal direction

**NB**: A pre-existing folder structure has already been created, so please stick to that folder structure, otherwise the scripts will not work.

In [1]:
%%writefile init_neb_k4_PW800.py
from os.path import join
from gpaw import GPAW, PW, FermiDirac
from ase.optimize import BFGS
from ase.parallel import paropen
from ase.io import read
from ase.visualize import view

folder = 'neb_k4_PW800/init'
folder = join(folder, '')  # Ensure trailing '/'

#parameters
#put here the formula of your perovskite, e.g. "CaGeO3"
name = "SrGeO3"

# The planewave cutoff, start with 600eV. 
# If you have time left when you are done with all calculation you can play around with this value to check convergence
ecut = 800
# Put in a number of k-points along one direction
# The total k-point grid will be (k,k,k)  
k = 4
k += k%2 # round up to next even number

atoms = read(f'relax/perovskite/{name}.traj')
atoms = atoms.repeat((2, 2, 2)) # 2 x 2 x 2 unit cell
del atoms[28]  # Make an oxygen vacancy

# We need to break the symmetry due to the introduction of the vacancy
atoms.rattle(0.05)

if 0:
    view(atoms)
    assert False

calc = GPAW(mode=PW(ecut),
            kpts={'size': (k, k, k),'gamma': True},
            xc='PBEsol',
            basis='dzp',
            txt=folder+'init.out',
            occupations=FermiDirac(width=0.01),
            symmetry={'point_group': False}  # Due to the rattle, we disable symmetries
            )

atoms.set_calculator(calc)

# Since we rattled the atoms quite a bit, we need to relax this tightly
BFGS(atoms, trajectory=folder+"init.traj",logfile=folder+"init.log").run(fmax=0.01)
epot = atoms.get_potential_energy()


with paropen(folder+'energy_init.dat', 'w') as f:
    print(f'Energy: {epot:.3f} eV', file=f)

Writing init_neb_k4_PW800.py


In [2]:
!qsub.py -p 16 -t 48 init_neb_k4_PW800.py

7462183.hnode2


### *y*-direction

In [3]:
%%writefile final_y_k4_PW800.py
from os.path import join
from gpaw import GPAW, PW, FermiDirac
from ase.optimize import BFGS
from ase.parallel import paropen
from ase.io import read
from gpaw import setup_paths
from ase.visualize import view


folder = 'neb_k4_PW800/final_y'
folder = join(folder, '')  # Ensure trailing '/'

#parameters
#put here the formula of your perovskite, e.g. "CaGeO3"
name = "SrGeO3"

# The planewave cutoff, start with 600eV. 
# If you have time left when you are done with all calculation you can play around with this value to check convergence
ecut = 800
# Put in a number of k-points along one direction
# The total k-point grid will be (k,k,k)  
k = 4
k += k%2 # round up to next even number

atoms = read(f'relax/perovskite/{name}.traj')
atoms = atoms.repeat((2, 2, 2)) # 2 x 2 x 2 unit cell
del atoms[38]

# We need to break the symmetry due to the introduction of the vacancy
atoms.rattle(0.05)

if 0:
    view(atoms)
    assert False

calc = GPAW(mode=PW(ecut),
            kpts={'size': (k, k, k),'gamma': True},
            xc='PBEsol',
            basis='dzp',
            txt=folder+'final_y.out',
            occupations=FermiDirac(width=0.01),
            symmetry={'point_group': False}  # Due to the rattle, we disable symmetries
            )

atoms.set_calculator(calc)

# Since we rattled the atoms quite a bit, we need to relax this tightly
BFGS(atoms, trajectory=folder+"final_y.traj",logfile=folder+"final_y.log").run(fmax=0.01)
epot = atoms.get_potential_energy()

with paropen(folder+'energy_y.dat', 'w') as f:
    print(f'Energy: {epot:.3f} eV', file=f)

Writing final_y_k4_PW800.py


In [4]:
!qsub.py -p 16 -t 48 final_y_k4_PW800.py

7462184.hnode2


Once you have constructed the initial and final images in the y-direction, we construct the path, and ensure that it looks reasonable.

**Remember to check that your path looks reasonable!**

Note: The below code should _not_ be submitted, but you can just run it directly.

In [1]:
from os.path import join
from ase.io import read, write
from ase import Atoms
from ase.visualize import view
from ase.neb import NEB

folder = 'neb_k4_PW800/neb_y'
folder = join(folder, '')

# Read initial and final states:
initial = read('neb_k4_PW800/init/init.traj')
final = read('neb_k4_PW800/final_y/final_y.traj')

# Make a band consisting of n images:
n = 7
m = n-2 # number of intermediate images

# Ensure that atom indices match
f_temp = final.copy()
new = Atoms()
new.set_cell(final.get_cell())
new.set_pbc(True)
for a in range(28):
    new.append(final[a])
for i in range(28, 37):
    new.append(final[i+1])
new.append(final[28])
for i in range(38, len(final)):
    new.append(final[i])
final = new.copy()
images = [initial]
images += [initial.copy() for i in range(m)]
images += [final]

neb = NEB(images)
neb.interpolate()

view(images)

# Write the path we constructed to be used by the NEB
write(folder+'initial_path.traj', images)

In [2]:
%%writefile neb_y_k4_PW800.py
from os.path import join
from gpaw import GPAW, PW, FermiDirac
from ase.optimize import FIRE
from ase.parallel import paropen
from ase.io import read
from ase.visualize import view
from ase.neb import NEB

folder = 'neb_k4_PW800/neb_y'
folder = join(folder, '')  # Ensure trailing '/'

#parameters
#put here the formula of your perovskite, e.g. "CaGeO3"
name = "SrGeO3"
# The planewave cutoff, start with 600eV. 
# If you have time left when you are done with all calculation you can play around with this value to check convergence
ecut = 800
# Put in a number of k-points along one direction
# The total k-point grid will be (k,k,k)  
k = 4
k += k%2 # round up to next even number

# load the path we wrote
images = read(folder+'initial_path.traj', index=':')

n = len(images)
m = n-2  # number of intermediate images
t = n-1

calc_final = GPAW(mode=PW(ecut),
                  kpts={'size': (k, k, k),'gamma': True},
                  xc='PBEsol',
                  basis='dzp',
                  txt=folder+'final.out',
                  #symmetry={'point_group':False}, # Use this if you get the 'symmetry broken' error
                  occupations=FermiDirac(width=0.01))

images[-1].set_calculator(calc_final)
images[-1].get_forces()  # Run the final image, so we get the energies again

neb = NEB(images)

# Set calculator:
for ii in range(1, t):
    image = images[ii]
    calc = GPAW(mode=PW(ecut),
            kpts={'size': (k,k,k),'gamma': True},
            xc='PBEsol',
            basis='dzp',
            txt=folder+f'neb-{ii}.out',
            #symmetry={'point_group':False}, #use this if you get the 'symmetry broken' error
            occupations=FermiDirac(width=0.01))
    image.set_calculator(calc)

optimizer = FIRE(neb, trajectory=folder+'neb.traj', logfile=folder+'neb.log')
optimizer.run(fmax=0.1)  # Do a course relaxation with regular NEB

# Switch on climbing image
cineb = NEB(images, climb=True)
optimizer = FIRE(cineb, trajectory=folder+'ci-neb.traj', logfile=folder+'ci-neb.log')
optimizer.run(fmax=0.05)

Writing neb_y_k4_PW800.py


In [3]:
!qsub.py -p 16 -t 96 neb_y_k4_PW800.py

7465970.hnode2


In [4]:
!qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
7465970.hnode2             ...y_k4_PW800.py s171629                0 R hpc            


In [ ]:
!ase gui neb_k6_PW800/neb_y/neb.traj@-7:

### Diagonal direction
We now construct the final image in the diagonal direction.

In [5]:
%%writefile final_diag_k4_PW800.py
from os.path import join
from gpaw import GPAW, PW, FermiDirac
from ase.optimize import BFGS
from ase.parallel import paropen
from ase.io import read

folder = 'neb_k4_PW800/final_diag'
folder = join(folder, '')  # Ensure trailing '/'

#parameters
#put here the formula of your perovskite, e.g. "CaGeO3"
name = "SrGeO3"

# The planewave cutoff, start with 600eV. 
# If you have time left when you are done with all calculation you can play around with this value to check convergence
ecut = 800
# Put in a number of k-points along one direction
# The total k-point grid will be (k,k,k)  
k = 4
k += k%2 # round up to next even number

atoms = read(f'relax/perovskite/{name}.traj')
atoms = atoms.repeat((2, 2, 2)) # 2 x 2 x 2 unit cell
del atoms[19]

# We need to break the symmetry due to the introduction of the vacancy
atoms.rattle(0.05)

if 0:
    view(atoms)
    assert False

calc = GPAW(mode=PW(ecut),
            kpts={'size': (k, k, k),'gamma': True},
            xc='PBEsol',
            basis='dzp',
            txt=folder+'final_diag.out',
            occupations=FermiDirac(width=0.01),
            symmetry={'point_group': False}  # Due to the rattle, we disable symmetries
            )

atoms.set_calculator(calc)

# Since we rattled the atoms quite a bit, we need to relax this tightly
BFGS(atoms, trajectory=folder+"final_diag.traj",logfile=folder+"final_diag.log").run(fmax=0.01)
epot = atoms.get_potential_energy()

with paropen(folder+'energy_y.dat', 'w') as f:
    print(f'Energy: {epot:.3f} eV', file=f)

Writing final_diag_k4_PW800.py


In [6]:
!qsub.py -p 16 -t 48 final_diag_k4_PW800.py

7462185.hnode2


Once again, once you have constructed the initial and final images in the diagonal direction, we construct the path, and ensure that it looks reasonable.

**Remember to check that your path looks reasonable!**

Note: The below code should _not_ be submitted, but you can just run it directly.

In [5]:
from os.path import join
from ase.io import read, write
from ase import Atoms
from ase.visualize import view
from ase.neb import NEB

folder = 'neb_k4_PW800/neb_diag'
folder = join(folder, '')

# Read initial and final states:
initial = read('neb_k4_PW800/init/init.traj')
final = read('neb_k4_PW800/final_diag/final_diag.traj')

# Make a band consisting of n images:
n = 7
m = n-2 # number of intermediate images

f_temp = final.copy()
new = Atoms()
new.set_cell(final.get_cell())
new.set_pbc(True)
for a in range(19):
    new.append(final[a])
new.append(final[27])
for i in range(19, 27):
    new.append(final[i])
for i in range(28,len(final)):
    new.append(final[i])
final = new.copy()

images = [initial]
images += [initial.copy() for i in range(m)]
images += [final]

neb = NEB(images)
neb.interpolate()

view(images)

# Write the path we constructed to be used by the NEB
write(folder+'initial_path.traj', images)

In [2]:
!pwd

/zhome/76/c/124557/energytools18/final_project/DFT


In [ ]:
!ase gui /zhome/76/c/124557/energytools18/final_project/DFT/neb/neb_diag/initial_path.traj

In [6]:
%%writefile neb_diag_k4_PW800.py
from os.path import join
from gpaw import GPAW, PW, FermiDirac
from ase.optimize import FIRE
from ase.parallel import paropen
from ase.io import read
from ase.neb import NEB

folder = 'neb_k4_PW800/neb_diag'
folder = join(folder, '')  # Ensure trailing '/'

#parameters
#put here the formula of your perovskite, e.g. "CaGeO3"
name = "SrGeO3"
# The planewave cutoff, start with 600eV. 
# If you have time left when you are done with all calculation you can play around with this value to check convergence
ecut = 800
# Put in a number of k-points along one direction
# The total k-point grid will be (k,k,k)  
k = 4
k += k%2 # round up to next even number

# load the path we wrote
images = read(folder+'initial_path.traj', index=':')
n = len(images)
m = n-2  # number of intermediate images
t = n-1

calc_final = GPAW(mode=PW(ecut),
                  kpts={'size': (k, k, k),'gamma': True},
                  xc='PBEsol',
                  basis='dzp',
                  txt=folder+'final.out',
                  #symmetry={'point_group':False}, # Use this if you get the 'symmetry broken' error
                  occupations=FermiDirac(width=0.01))
images[-1].set_calculator(calc_final)
images[-1].get_forces()  # Run the final image, so we get the energies again

neb = NEB(images)

# Set calculator:
for ii in range(1, t):
    image = images[ii]
    calc = GPAW(mode=PW(ecut),
            kpts={'size': (k, k, k),'gamma': True},
            xc='PBEsol',
            basis='dzp',
            txt=folder+f'neb-{ii}.out',
            #symmetry={'point_group':False}, #use this if you get the 'symmetry broken' error
            occupations=FermiDirac(width=0.01))
    image.set_calculator(calc)

optimizer = FIRE(neb, trajectory=folder+'neb.traj', logfile=folder+'neb.log')
optimizer.run(fmax=0.1)  # Do a course relaxation with regular NEB

# Switch on climbing image
cineb = NEB(images, climb=True)
optimizer = FIRE(cineb, trajectory=folder+'ci-neb.traj', logfile=folder+'ci-neb.log')
optimizer.run(fmax=0.05)

Writing neb_diag_k4_PW800.py


In [7]:
!qsub.py -p 16 -t 96 neb_diag_k4_PW800.py

7465972.hnode2


In [8]:
!qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
7465970.hnode2             ...y_k4_PW800.py s171629                0 R hpc            
7465972.hnode2             ...g_k4_PW800.py s171629                0 R hpc            


In [3]:
!ase gui neb/neb_diag/ci-neb.traj@-7: